## Campbell e Schiller (1991)

Usando dados das LTNs

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [2]:
# mostra o dataframe inteiro
# pd.set_option('display.max_rows', None)

Para o caso brasileiro, a base de dados que será usada são os preços da Letra do Tesouro Nacional (LTN), cujo pagamento de R\$ 1000 é feito após $t$ períodos. As LTNs farão o papel dos títulos americanos.

Os dados das LTNs estão em periodicidade mensal (último dia do mês), e abrangem todo o período a partir de Junho de 2012 até Outubro de 2022. Essa base de dados também foi obtida via Reuters com acesso concedido pelo departamento de Economia da PUC-Rio.

In [3]:
# lendo base de dados dos preços das LTNs
ltn_precos = pd.read_csv('ltn_precos.csv', sep=',', index_col=0)

In [4]:
# função que transforma a coluna de datas em datas de fato
ltn_precos.index = pd.to_datetime(ltn_precos.index)

In [5]:
# dropando colunas que não serão usadas
ltn_precos = ltn_precos.drop(['BR6MT=RR', 'Unnamed: 5', 'Unnamed: 6'], axis=1)

In [6]:
# dropando valores NaNs
ltn_precos = ltn_precos.loc[ltn_precos.index.dropna()]

In [7]:
# normalizando preços das LTNs para pagamento de R$1
ltn_precos = ltn_precos/1000

In [8]:
# base de dados pronta!
ltn_precos

,BR1YT=RR,BR2YT=RR,BR3YT=RR
2022-10-31,0.888250,0.829531,0.996378
2022-09-30,0.908698,0.826359,0.991628
2022-08-31,0.898608,0.815204,0.976025
2022-07-31,0.887308,0.800305,0.952697
2022-06-30,0.878349,0.792686,0.990984
...,...,...,...
2012-10-31,0.938520,0.850100,1.083364
2012-09-30,0.930100,0.873745,1.058065
2012-08-31,0.924305,0.866180,1.048780
2012-07-31,0.937410,0.860395,1.042515


Temos que $BRnYT=RR$ é o preço da LTN em $t$ que paga R\$1 em $t+n$. Portando podemos usar a seguinte notação

$$
B_t^{(n)} = B_t^{t\rightarrow t+n} = BRnYT=RR
$$

In [10]:
# renomeando as colunas de acordo
ltn_precos = ltn_precos.rename(columns={'BR1YT=RR':'B_t(1)', 'BR2YT=RR':'B_t(2)', 'BR3YT=RR':'B_t(3)'})

Logo o retorno anualizado da LTN da obtido será

$$
(1+r_t^{(n)})^n = \frac{B_{t+n}^{(0)}}{B_t^{(n)}}
$$

Como $B_{t+n}^{(0)}$ é o preço da LTN comprada em $t+n$ que entrega R\$1 em $t+n$, seu valor deve ser 1. Portanto

$$
r_t^{(n)} = \left(\frac{1}{B_t^{(n)}}\right)^{(\frac{1}{n})} -1
$$

In [11]:
ltn_precos['r_t(1)'] = (1/ltn_precos['B_t(1)']) - 1
ltn_precos['r_t(2)'] = (1/ltn_precos['B_t(2)'])**(1/2) - 1
ltn_precos['r_t(3)'] = (1/ltn_precos['B_t(3)'])**(1/3) - 1

Definindo $P_t^{(n)} = \ln[B_t^{(n)}]$ e $y_t^{(n)} = \ln[1+r_t^{(n)}]$

In [12]:
ltn_precos['P_t(1)'] = np.log(ltn_precos['B_t(1)'])
ltn_precos['P_t(2)'] = np.log(ltn_precos['B_t(2)'])
ltn_precos['P_t(3)'] = np.log(ltn_precos['B_t(3)'])

In [13]:
ltn_precos['y_t(1)'] = np.log(1+ltn_precos['r_t(1)'])
ltn_precos['y_t(2)'] = np.log(1+ltn_precos['r_t(2)'])
ltn_precos['y_t(3)'] = np.log(1+ltn_precos['r_t(3)'])

Temos que 

$$
y_t^{(n)} = -\frac{P_t^{(n)}}{n}
$$

In [14]:
ltn_precos

,B_t(1),B_t(2),B_t(3),r_t(1),r_t(2),r_t(3),P_t(1),P_t(2),P_t(3),y_t(1),y_t(2),y_t(3)
2022-10-31,0.888250,0.829531,0.996378,0.125809,0.097953,0.001210,-0.118502,-0.186895,-0.003629,0.118502,0.093447,0.001210
2022-09-30,0.908698,0.826359,0.991628,0.100476,0.100058,0.002806,-0.095742,-0.190726,-0.008407,0.095742,0.095363,0.002802
2022-08-31,0.898608,0.815204,0.976025,0.112832,0.107559,0.008122,-0.106908,-0.204317,-0.024267,0.106908,0.102158,0.008089
2022-07-31,0.887308,0.800305,0.952697,0.127004,0.117821,0.016284,-0.119563,-0.222762,-0.048458,0.119563,0.111381,0.016153
2022-06-30,0.878349,0.792686,0.990984,0.138500,0.123180,0.003024,-0.129711,-0.232328,-0.009057,0.129711,0.116164,0.003019
...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-31,0.938520,0.850100,1.083364,0.065507,0.084588,-0.026337,-0.063451,-0.162401,0.080071,0.063451,0.081201,-0.026690
2012-09-30,0.930100,0.873745,1.058065,0.075153,0.069812,-0.018638,-0.072463,-0.134967,0.056442,0.072463,0.067483,-0.018814
2012-08-31,0.924305,0.866180,1.048780,0.081894,0.074474,-0.015751,-0.078713,-0.143663,0.047628,0.078713,0.071831,-0.015876
2012-07-31,0.937410,0.860395,1.042515,0.066769,0.078080,-0.013783,-0.064635,-0.150364,0.041636,0.064635,0.075182,-0.013879


Precisamos definir a yield $y_{t+1}^{(n-1)}$ como o rendimento de comprar uma LTN em $t+1$ que paga R\$1 em $t+1+(n-1) = t+n$

In [15]:
ltn_precos['y_t+1(2-1)'] = ltn_precos['y_t(1)'].shift(12)
ltn_precos['y_t+1(3-1)'] = ltn_precos['y_t(2)'].shift(12)

Defina $s_t^{(n)} = y_t^{(n)} - y_t^{(1)}$

In [16]:
ltn_precos['s_t(2)'] = ltn_precos['y_t(2)'] - ltn_precos['y_t(1)']
ltn_precos['s_t(3)'] = ltn_precos['y_t(3)'] - ltn_precos['y_t(1)']

O primeiro modelo de Campbell e Schiller (1991) é mostrado a seguir

$$
y_{t+1}^{(n-1)} - y_t^{(n)} = \alpha_n + \beta_n \frac{s_t^{(n)}}{n-1} + \epsilon_t^{(n)}
$$

In [17]:
ltn_precos['y'] = ltn_precos['y_t+1(2-1)'] - ltn_precos['y_t(2)']
ltn_precos['x'] = ltn_precos['s_t(2)']
result = sm.ols(formula="y ~ x", data=ltn_precos).fit()

Para $n = 2$, a regressão é

$$
y_{t+1}^{(1)} - y_t^{(2)} = \alpha_2 + \beta_2 s_t^{(2)} + \epsilon_t^{(2)}
$$

In [18]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     11.44
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           0.000996
Time:                        11:53:36   Log-Likelihood:                 233.67
No. Observations:                 113   AIC:                            -463.3
Df Residuals:                     111   BIC:                            -457.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0004      0.003     -0.114      0.9

In [19]:
ltn_precos['y'] = ltn_precos['y_t+1(3-1)'] - ltn_precos['y_t(3)']
ltn_precos['x'] = ltn_precos['s_t(3)']/2
result = sm.ols(formula="y ~ x", data=ltn_precos).fit()

Para $n = 3$, a regressão é

$$
y_{t+1}^{(2)} - y_t^{(3)} = \alpha_3 + \beta_3 \frac{s_t^{(3)}}{2} + \epsilon_t^{(3)}
$$

In [20]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     331.3
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           4.13e-35
Time:                        11:53:51   Log-Likelihood:                 221.22
No. Observations:                 113   AIC:                            -438.4
Df Residuals:                     111   BIC:                            -433.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0083      0.004      2.315      0.0